In [1]:
import os
import pprint
import operator
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances 

In [2]:
#Information about the data
info = pd.read_csv('Data ml-100k/u.info', sep=" ", header=None)
info.columns=['Counts', 'Type']
info

,Counts,Type
0,943,users
1,1682,items
2,100000,ratings


In [5]:
#Types of genres
genre = pd.read_csv('Data ml-100k/u.genre', sep="|", encoding='latin-1', header=None)
genre.drop(genre.columns[1], axis=1, inplace=True)
genre.columns = ['Genres']
genre_list = list(genre['Genres'])
genre_list

['unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [6]:
#Types of occupations
occupation = pd.read_csv('Data ml-100k/u.occupation', sep="|", encoding='latin-1', header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
occupation_list

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [10]:
#Ratings data
data = pd.read_csv('Data ml-100k/u.data', sep="\t", header=None)
data.columns = ['user id', 'movie id', 'rating', 'timestamp']
data.head()

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
#Movies data
item = pd.read_csv('Data ml-100k/u.item', sep="|", encoding='latin-1', header=None)
item.columns = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 
                'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
#User data
user = pd.read_csv('Data ml-100k/u.user', sep="|", encoding='latin-1', header=None)
user.columns = ['user id', 'age', 'gender', 'occupation', 'zip code']
user.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


**TOP 3 MOVIES BY OCCUPATION**

In [11]:
#merge the data table with user table
data_user = pd.merge(data[['user id', 'movie id', 'rating']], user[['user id', 'occupation']], on='user id')
data_user.drop(columns = ['user id'], inplace=True)

#merge the Data_User dataframe with Item dataframe to get each rating, occupation of user and movie title
data_user_item = pd.merge(data_user[['movie id', 'rating', 'occupation']], item[['movie id', 'movie title']], on='movie id')
data_user_item.drop(columns = ['movie id'], inplace=True)

#group the data by occupation and movie title and sort with decreasing average ratings
data_user_item_sorted = data_user_item.groupby(['occupation', 'movie title'], as_index=False)['rating'].mean().sort_values('rating', ascending=False)

#group data by occupation, movie title and select top 3 movies for each occupation
top_3_occ = data_user_item_sorted.groupby(['occupation']).head(3).sort_values(['occupation', 'movie title'], ascending=[True, True]).reset_index()
top_3_occ.drop(['index'], axis=1, inplace=True)
# top_3_occ.to_csv('Top3ByOccupation.csv', index=False, sep=',')
top_3_occ

,occupation,movie title,rating
0,administrator,When We Were Kings (1996),5.0
1,administrator,"Winter Guest, The (1997)",5.0
2,administrator,"World of Apu, The (Apur Sansar) (1959)",5.0
3,artist,"39 Steps, The (1935)",5.0
4,artist,"Adventures of Pinocchio, The (1996)",5.0
...,...,...,...
58,technician,Basquiat (1996),5.0
59,technician,Beautiful Thing (1996),5.0
60,writer,Faster Pussycat! Kill! Kill! (1965),5.0
61,writer,Faust (1994),5.0


Note: Some occupations have multiple movies that are top rated (5 stars). In that case the movies have been sorted by the titles and the top 3 are obtained.

**TOP 3 MOVIES BY GENRE**

In [13]:
#merge the data table with the item table
data_item = pd.merge(data[['user id', 'movie id', 'rating']], item, on='movie id')
data_item.drop(columns = ['user id', 'movie id', 'release date', 'video release date', 'IMDb URL'], inplace=True)

#For each genre get the top 3 movies by average rating
top_3_genre = pd.DataFrame()
for gen in genre_list:
    g_r = data_item[data_item[gen] == 1]
    new_gen = pd.DataFrame(g_r.groupby(['movie title'], as_index=False)['rating'].mean().sort_values(['rating', 'movie title'], ascending=[False, True]).head(3))
    new_gen.insert(0, 'genre', gen)
    top_3_genre = top_3_genre.append(new_gen, ignore_index=True)

# top_3_genre.to_csv('Top3ByGenre.csv', index=False, sep=',')
top_3_genre

,genre,movie title,rating
0,unknown,unknown,3.444444
1,unknown,Good Morning (1971),1.000000
2,Action,Star Wars (1977),4.358491
3,Action,"Godfather, The (1972)",4.283293
4,Action,Raiders of the Lost Ark (1981),4.252381
5,Adventure,Star Kid (1997),5.000000
6,Adventure,Star Wars (1977),4.358491
7,Adventure,Raiders of the Lost Ark (1981),4.252381
8,Animation,"Close Shave, A (1995)",4.491071
9,Animation,"Wrong Trousers, The (1993)",4.466102


**TOP 3 MOVIES BY OCCUPATION, GENRE**

In [16]:
#merge the data table with user table
data_user_og = pd.merge(data[['user id', 'movie id', 'rating']], user[['user id', 'occupation']], on='user id')
data_user_og.drop(columns = ['user id'], inplace=True)

#merge the Data_User dataframe with Item dataframe to get each rating, occupation of user and movie title
data_user_item_og = pd.merge(data_user_og[['movie id', 'rating', 'occupation']], item, on='movie id')
data_user_item_og.drop(columns = ['movie id'], inplace=True)

top_3_occ_genre = pd.DataFrame()
for occ in list(occupation['Occupations']):
    occ_table = data_user_item_og[data_user_item_og['occupation']==occ]
    for gen in genre_list:
        g_o_r = occ_table[occ_table[gen] == 1]
        new_occ_gen = pd.DataFrame(g_o_r.groupby(['movie title'], as_index=False)['rating'].mean().sort_values(['rating', 'movie title'], ascending=[False, True]).head(3))
        new_occ_gen.insert(0, 'genre', gen)
        new_occ_gen.insert(0, 'occupation', occ)
        top_3_occ_genre = top_3_occ_genre.append(new_occ_gen, ignore_index=True)
        
# top_3_occ_genre.to_csv('Top3ByOcc&Genre.csv', index=False, sep=',')
top_3_occ_genre

,occupation,genre,movie title,rating
0,administrator,Action,Congo (1995),5.000000
1,administrator,Action,"Magnificent Seven, The (1954)",4.583333
2,administrator,Action,"Princess Bride, The (1987)",4.565217
3,administrator,Adventure,"Big Blue, The (Grand bleu, Le) (1988)",5.000000
4,administrator,Adventure,Congo (1995),5.000000
...,...,...,...,...
1132,writer,War,Henry V (1989),4.444444
1133,writer,War,"Killing Fields, The (1984)",4.333333
1134,writer,Western,Dead Man (1995),4.500000
1135,writer,Western,"Good, The Bad and The Ugly, The (1966)",4.000000


**TOP 3 MOVIES BY AGE**

In [15]:
#create a column of age group in the users dataframe
bins= [0,6,12,18,30,50,200]
labels = ['<=6','<=12','<=18','<=30','<=50', '50+']
user['age group'] = pd.cut(user['age'], bins=bins, labels=labels, right=True)

#merge the data table with the user table
data_user_age = pd.merge(data[['user id', 'movie id', 'rating']], user[['user id', 'age group']], on='user id')
data_user_age.drop(columns = ['user id'], inplace=True)

#merge the data_user_age table with the item table to get each rating, age group of user and movie title
data_user_item_age = pd.merge(data_user_age[['movie id', 'rating', 'age group']], item[['movie id', 'movie title']], on='movie id')
data_user_item_age.drop(columns = ['movie id'], inplace=True)
data_user_item_age['age group'] = data_user_item_age['age group'].astype('category')

#group the data by age group and movie title and sort with decreasing average ratings
data_user_item_age_sorted = data_user_item_age.groupby(['age group', 'movie title'], as_index=False)['rating'].mean().sort_values('rating', ascending=False)

#group data by occupation, movie title and select top 3 movies for each occupation
top_3_age = data_user_item_age_sorted.groupby(['age group']).head(3).sort_values(['age group', 'movie title'], ascending=[True, True]).reset_index()
top_3_age.drop(['index'], axis=1, inplace=True)
# top_3_age.to_csv('Top3ByAge.csv', index=False, sep=',')
top_3_age

,age group,movie title,rating
0,<=6,'Til There Was You (1997),NaN
1,<=6,1-900 (1994),NaN
2,<=6,101 Dalmatians (1996),NaN
3,<=12,101 Dalmatians (1996),5.0
4,<=12,"English Patient, The (1996)",5.0
5,<=12,Waiting for Guffman (1996),5.0
6,<=18,Laura (1944),5.0
7,<=18,Living in Oblivion (1995),5.0
8,<=18,Looking for Richard (1996),5.0
9,<=30,Maya Lin: A Strong Clear Vision (1994),5.0


We observe here that no movie has been rated in the age group having ages less than 6, hence we have the movie titles sorted by titles for this group in the results. For the other age groups, we have the top 3 moveis having the highest average rating in that age group sorted by movie titles.

**TOP 3 GENRES RELEASED IN SUMMER [MAY-JULY]**

To look at the top 3 genres released during the months of summer, we will get the top 3 genres based on the number of movies as well as the average ratings for movies having these genres.

1) By count

In [17]:
#merge data table with item table and count movies by genre
data_item_gen = pd.merge(data[['user id', 'movie id', 'rating']], item, on='movie id')
data_item_gen['release date 2'] = pd.to_datetime(data_item_gen['release date'])
data_item_gen['release month'] = data_item_gen['release date 2'].dt.month
data_item_gen_summer = data_item_gen[(data_item_gen['release month']>=5) & (data_item_gen['release month']<=7)]

count_gen = {}
for gen in genre_list:
    count_gen[gen] = sum(data_item[gen])

sorted_count_gen = sorted(count_gen.items(), key=operator.itemgetter(1), reverse=True)
sorted_count_gen[:3]

[('Drama', 39895), ('Comedy', 29832), ('Action', 25589)]

2) By average rating

In [18]:
top_3_genre_summer = pd.DataFrame(columns = ['genre', 'average rating'])
for gen in genre_list:
    genre_this = data_item_gen_summer[data_item_gen_summer[gen] == 1]
    row = [gen, genre_this['rating'].mean()]
    top_3_genre_summer.loc[len(top_3_genre_summer)] = row
    
top_3_genre_summer_res = top_3_genre_summer.sort_values('average rating', ascending=False).head(3)
# top_3_genre_summer_res.to_csv('Top3GenresSummer.csv', index=False, sep=',')
top_3_genre_summer_res

,genre,average rating
18,Western,3.823529
8,Drama,3.654290
13,Mystery,3.516071


**TOP 2 CO-OCCURRING GENRES FOR EACH GENRE**

In [19]:
#merge the data table with the item table
data_item_gen_co = pd.merge(data[['user id', 'movie id', 'rating']], item, on='movie id')

#for each genre, calculate the top 2 co-occurring genres and store results in a dictionary
top2gens = {}
for gen1 in genre_list:
    t = {}
    for gen2 in genre_list:
        if gen1 != gen2:
            t[gen2] = data_item_gen_co[(data_item_gen_co[gen1]==1) & (data_item_gen_co[gen2]==1)].shape[0]
    sorted_t = sorted(t.items(), key=operator.itemgetter(1), reverse=True)
    top2gens[gen1] = sorted_t[:2]

# fout = "Top2CooccurringGen.txt"
# fo = open(fout, "w")
# for k, v in top2gens.items():
#     fo.write(str(k) + ' >>> '+ str(v) + '\n')
# fo.close()
top2gens

{'unknown': [('Action', 0), ('Adventure', 0)],
 'Action': [('Adventure', 10305), ('Thriller', 10100)],
 'Adventure': [('Action', 10305), ('Sci-Fi', 5135)],
 'Animation': [("Children's", 2932), ('Musical', 1868)],
 "Children's": [('Comedy', 3122), ('Animation', 2932)],
 'Comedy': [('Romance', 7542), ('Drama', 4117)],
 'Crime': [('Drama', 4067), ('Thriller', 3157)],
 'Documentary': [('Drama', 57), ('War', 53)],
 'Drama': [('Romance', 8017), ('Thriller', 5419)],
 'Fantasy': [("Children's", 807), ('Sci-Fi', 656)],
 'Film-Noir': [('Thriller', 974), ('Mystery', 766)],
 'Horror': [('Thriller', 1811), ('Action', 1285)],
 'Musical': [("Children's", 2492), ('Animation', 1868)],
 'Mystery': [('Thriller', 3270), ('Drama', 1343)],
 'Romance': [('Drama', 8017), ('Comedy', 7542)],
 'Sci-Fi': [('Action', 7968), ('Adventure', 5135)],
 'Thriller': [('Action', 10100), ('Drama', 5419)],
 'War': [('Drama', 5162), ('Action', 4527)],
 'Western': [('Action', 848), ('Comedy', 568)]}

**FOR EACH USER FIND ANOTHER ONE WITH SIMILAR PREFREANCES**

In [20]:
#create a user-rating matrix
data_matrix = np.zeros((info['Counts'][0], info['Counts'][1]))
for line in data.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

#calculate user similarity matrix based on cosine similarity of ratings
user_similarity = pairwise_distances(data_matrix, metric='cosine')

#create dictionary of each user and user with most simiar preferances
sim_user= {}
for i in range(943):
    sim_user[i+1] = [np.argmax(user_similarity[i])+1]

# fout = "MostSimUser.txt"
# fo = open(fout, "w")    
# for k, v in sim_user.items():
#     fo.write(str(k) + ' >>> '+ str(v) + '\n\n')
# fo.close()